In [1]:
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F

## 5.1 Layers and Blocks

In [2]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
print(X)
net(X)

tensor([[0.2261, 0.9156, 0.9515, 0.8078, 0.7337, 0.1514, 0.3194, 0.2247, 0.5511,
         0.0674, 0.2560, 0.4592, 0.8733, 0.8491, 0.9222, 0.4523, 0.1675, 0.7597,
         0.8442, 0.2870],
        [0.7024, 0.7475, 0.9076, 0.9373, 0.5916, 0.2318, 0.6411, 0.3997, 0.7004,
         0.0085, 0.6811, 0.2387, 0.4854, 0.3865, 0.1997, 0.5945, 0.9104, 0.8442,
         0.5445, 0.6459]])


tensor([[-0.1455, -0.0394,  0.1052,  0.1665,  0.1212,  0.0408, -0.0374,  0.3147,
          0.1025,  0.0050],
        [-0.2289, -0.1541,  0.0804,  0.0544,  0.0713,  0.2024,  0.0460,  0.3371,
          0.0218, -0.0120]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
    '''
    Notice how we are using forward
    '''
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
mlp = MLP()
mlp(X)

tensor([[ 0.0433, -0.1173,  0.0062, -0.1537, -0.2294, -0.0030,  0.0260,  0.2941,
          0.0409,  0.0840],
        [ 0.1075, -0.1261,  0.0516, -0.1063, -0.2502,  0.0721,  0.0884,  0.2122,
          0.1120,  0.0710]], grad_fn=<AddmmBackward>)

In [5]:
class Mysequential(nn.Module):
    '''
    Implementation of sequential class
    '''
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
myseq = Mysequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
myseq(X)

tensor([[ 0.0143,  0.0818, -0.1238,  0.1066, -0.2620, -0.0147, -0.0515,  0.0759,
         -0.2427, -0.1506],
        [-0.0660,  0.0889, -0.2186,  0.0911, -0.2124, -0.0109,  0.0808,  0.0219,
         -0.1001, -0.1909]], grad_fn=<AddmmBackward>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0558, grad_fn=<SumBackward0>)

## 5.2 Parameter Managemnt

In [8]:
X = torch.rand(2,4)
print(X)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
net(X)

tensor([[0.7631, 0.4118, 0.6470, 0.4275],
        [0.4130, 0.6801, 0.6364, 0.3697]])


tensor([[-0.1454],
        [-0.0834]], grad_fn=<AddmmBackward>)

In [9]:
print(net[2].state_dict())
print(net[1].state_dict())

OrderedDict([('weight', tensor([[-0.2397,  0.1773,  0.2084,  0.2019,  0.3323,  0.1223, -0.2514, -0.1569]])), ('bias', tensor([-0.0553]))])
OrderedDict()


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0553], requires_grad=True)
tensor([-0.0553])


In [11]:
net[2].weight.grad == None

True

In [12]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data

tensor([-0.0553])

In [14]:
def block1():
    net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
    return net
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
rgnet = nn.Sequential(block2(),nn.Linear(4,1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0061, -0.0152, -0.0108,  0.0090]), tensor(0.))

In [16]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [17]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)== nn.Linear:
        nn.init.constant_(m.weight,42)
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0678, -0.4659, -0.2431,  0.4241])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [18]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000, -8.4026, -0.0000],
        [ 8.2239, -0.0000,  7.2218, -7.2940]], grad_fn=<SliceBackward>)

In [19]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -7.4026,  1.0000])

In [20]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [21]:
import torch
from torch import nn
import torch.nn.functional as F

In [30]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,X):
        return X - X.mean()

X = torch.FloatTensor([1,2,3,4])
layer = CenteredLayer()
layer(X)

tensor([-1.5000, -0.5000,  0.5000,  1.5000])

In [37]:
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())
X = torch.randn(4,8)
Y = net(X)
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

In [38]:
torch.cuda.device_count()

0

In [39]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])